## **Bioinformatics Toolkit: Data Upload and Setup**

**This notebook provides an interactive interface for uploading, inspecting, and storing bioinformatics data. By the end of this notebook, you will have loaded your CSV data into a SQLite database and ensured its integrity.**


In [2]:
# Necessary imports for our notebook operations.
# 'pandas' for data manipulation and 'database' for our SQLite operations.

import sys
sys.path.append("../")

import pandas as pd
from src import database
import datetime

In [3]:
# Create the SQLite database connection
conn = database.create_connection()

# Create the genetic_data table with the provided schema
create_table_sql = """
CREATE TABLE IF NOT EXISTS genetic_data (
    id INTEGER PRIMARY KEY AUTOINCREMENT, -- This auto-increments and ensures each row has a unique ID
    rsid_key TEXT NOT NULL,
    clean_variant_name_fixed TEXT NOT NULL,
    clean_pip_score REAL,
    clean_p_value REAL,
    clean_pip_score_filter TEXT,
    sequence_250bp TEXT,
    extracted_ref TEXT,
    original_ref TEXT,
    upload_timestamp DATETIME DEFAULT CURRENT_TIMESTAMP -- This adds the current timestamp when a row is inserted
);
"""

database.create_table(conn, create_table_sql)


In [5]:

# Load the CSV data
csv_path = "../data/sample_data.csv"
data = pd.read_csv(csv_path)

# Add a timestamp column with the current date and time
data['upload_timestamp'] = datetime.datetime.now()

# Display the data to inspect the new column
data.head()

,rsid_key,clean_variant_name_fixed,clean_pip_score,clean_p_value,clean_pip_score_filter,sequence_250bp,extracted_ref,original_ref,upload_timestamp
0,rs6600249,chr1:24978669:T:C,0.020849,NaN,True,ATGTACCACCTCTTGTCTGGTGGTGTGAGCAACTCATTCCCCACCT...,T,T,2023-09-16 00:11:13.557774
1,rs4649040,chr1:24967411:C:T,0.003881,NaN,True,CCCCCCCAAGCCCCTATAACTTGATCACCCTGAGGGGCTTCCCCCT...,C,C,2023-09-16 00:11:13.557774
2,rs4648889,chr1:24967338:G:A,0.003881,NaN,True,caccaacaccgcctccAGTTTTGAACATACCACCCTACGTGCTTTG...,G,G,2023-09-16 00:11:13.557774
3,rs4265380,chr1:24966865:C:T,0.003881,NaN,True,ctctgggcctcagttttcttccctgtaaaatggggataacaccttc...,C,C,2023-09-16 00:11:13.557774
4,rs6600250,chr1:24978681:C:T,0.020849,NaN,True,TTGTCTGGTGGTGTGAGCAACTCATTCCCCACCTCTCATCCAGATA...,C,C,2023-09-16 00:11:13.557774


In [6]:
# Check for missing values in each column
missing_values = data.isnull().sum()

# Check for duplicate rsid_key values in the CSV data
duplicate_rsid_keys = data[data.duplicated(subset="rsid_key", keep=False)]

if not duplicate_rsid_keys.empty:
    print("Warning: Found duplicate rsid_key values in the CSV data!")
    display(duplicate_rsid_keys)
else:
    print("No duplicate rsid_key values found.")


No duplicate rsid_key values found.


## Store the Data in SQLite Database


In [7]:
# Insert the data into the genetic_data table
# Note: The 'if_exists' parameter determines what to do if the table already exists. 
# 'replace' will replace existing data, while 'append' will insert new rows.
data.to_sql('genetic_data', conn, if_exists='append', index=False)

692

## Verify Database Upload


In [8]:
# Fetch the first few rows from the database to verify
query = "SELECT * FROM genetic_data LIMIT 5;"
pd.read_sql(query, conn)


,id,rsid_key,clean_variant_name_fixed,clean_pip_score,clean_p_value,clean_pip_score_filter,sequence_250bp,extracted_ref,original_ref,upload_timestamp
0,1,rs6600249,chr1:24978669:T:C,0.020849,None,1,ATGTACCACCTCTTGTCTGGTGGTGTGAGCAACTCATTCCCCACCT...,T,T,2023-09-16 00:11:13.557774
1,2,rs4649040,chr1:24967411:C:T,0.003881,None,1,CCCCCCCAAGCCCCTATAACTTGATCACCCTGAGGGGCTTCCCCCT...,C,C,2023-09-16 00:11:13.557774
2,3,rs4648889,chr1:24967338:G:A,0.003881,None,1,caccaacaccgcctccAGTTTTGAACATACCACCCTACGTGCTTTG...,G,G,2023-09-16 00:11:13.557774
3,4,rs4265380,chr1:24966865:C:T,0.003881,None,1,ctctgggcctcagttttcttccctgtaaaatggggataacaccttc...,C,C,2023-09-16 00:11:13.557774
4,5,rs6600250,chr1:24978681:C:T,0.020849,None,1,TTGTCTGGTGGTGTGAGCAACTCATTCCCCACCTCTCATCCAGATA...,C,C,2023-09-16 00:11:13.557774


The data has been successfully uploaded to the SQLite database. Make sure to thoroughly inspect and validate the data.
